# Colab code

In [1]:
!apt install subversion

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 subversion
0 upgraded, 5 newly installed, 0 to remove and 14 not upgraded.
Need to get 2,237 kB of archives.
After this operation, 9,910 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libapr1 amd64 1.6.3-2 [90.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libaprutil1 amd64 1.6.1-2 [84.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libserf-1-1 amd64 1.3.9-6 [44.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libsvn1 amd64 1.9.7-4ubuntu1 [1,183 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 subversion amd64 1.9.7-4ubuntu1 

In [2]:
!svn checkout https://github.com/sbettid/ATML_Project_202021

Output streaming troncato alle ultime 5000 righe.
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05434_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05435_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05436_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05437_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05438_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05439_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05440_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05441_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05442_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05443_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05444_1_1_1_0_0_01.png
A    ATML_Project_202021/trunk/mrlEyes_2018_01/1/s0036_05445_1_1_1_0_0_01.png
A    ATML_Proj

In [3]:
if 'google.colab' in str(get_ipython()):
  # connect to the datasets
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  closed_eyes_dataset_path = "/content/ATML_Project_202021/trunk/mrlEyes_2018_01"
else:
  closed_eyes_dataset_path = "mrlEyes_2018_01"

Mounted at /content/drive


# Import required libraries

In [4]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

# Dataset preparation and pre-processing all in one

Reading the dataset

In [ ]:
images = []
labels = []
path = closed_eyes_dataset_path + "/s00"
for i in range (1,10):
  if i < 10:
    curr_path = path + "0" + str(i)
  else:
    curr_path = path + str(i)
  print("Analysing folder: ", curr_path)
  for filename in os.listdir(curr_path):
    img = cv2.imread(curr_path + "/" + filename)
    images.append(cv2.resize(img,(80,80), interpolation=cv2.INTER_AREA)/255)
    labels.append(int(filename.split("_")[4]))

Analysing folder:  /content/ATML_Project_202021/trunk/mrlEyes_2018_01/s0001
Analysing folder:  /content/ATML_Project_202021/trunk/mrlEyes_2018_01/s0002
Analysing folder:  /content/ATML_Project_202021/trunk/mrlEyes_2018_01/s0003
Analysing folder:  /content/ATML_Project_202021/trunk/mrlEyes_2018_01/s0004
Analysing folder:  /content/ATML_Project_202021/trunk/mrlEyes_2018_01/s0005
Analysing folder:  /content/ATML_Project_202021/trunk/mrlEyes_2018_01/s0006
Analysing folder:  /content/ATML_Project_202021/trunk/mrlEyes_2018_01/s0007
Analysing folder:  /content/ATML_Project_202021/trunk/mrlEyes_2018_01/s0008
Analysing folder:  /content/ATML_Project_202021/trunk/mrlEyes_2018_01/s0009


## Splitting the dataset

In [ ]:
X_train, X_rest, y_train, y_rest = train_test_split(images, labels, test_size = 0.3)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size = 0.5)

## Test

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [40]:
train_generator = datagen.flow_from_directory(closed_eyes_dataset_path,target_size=(80,80), subset="training", class_mode="binary")

Found 67919 images belonging to 2 classes.


In [39]:
val_generator = datagen.flow_from_directory(closed_eyes_dataset_path,target_size=(80,80), subset="validation", class_mode="binary")

Found 16979 images belonging to 2 classes.


In [41]:
train_generator.class_indices

{'0': 0, '1': 1}

In [42]:
val_generator.class_indices

{'0': 0, '1': 1}

## Importing pre-trained model

In [10]:
from tensorflow.keras.applications.xception import Xception

In [22]:
xmodel = Xception(input_shape = (80,80,3), include_top = False, weights = "imagenet", pooling="max")

In [12]:
layers_num = len(xmodel.layers)

In [23]:
i = 1

for layer in xmodel.layers:
    if layers_num - i <= 5:
        layer.trainable = True
    else:
        layer.trainable = False
    i += 1

In [25]:
xmodel.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 39, 39, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 39, 39, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 39, 39, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [43]:
last_layer = xmodel.get_layer('global_max_pooling2d_1')
last_output = last_layer.output

our_layer = keras.layers.Flatten()(last_output)
our_layer = keras.layers.Dense(512, activation='relu')(our_layer)
our_layer = keras.layers.Dropout(0.5)(our_layer)
our_layer = keras.layers.Dense(256, activation='relu')(our_layer)
our_layer = keras.layers.Dropout(0.3)(our_layer)
our_layer = keras.layers.Dense(1, activation='sigmoid')(our_layer)

In [44]:
model = keras.Model(xmodel.input,our_layer)

In [28]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 39, 39, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 39, 39, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 39, 39, 32)   0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [45]:
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [46]:
history = model.fit(train_generator, validation_data=val_generator, epochs = 5)

Epoch 1/5
2123/2123 [==============================] - 160s 75ms/step - loss: 0.0308 - accuracy: 0.9896 - val_loss: 0.4097 - val_accuracy: 0.9221
Epoch 2/5
2123/2123 [==============================] - 160s 75ms/step - loss: 0.0217 - accuracy: 0.9922 - val_loss: 0.6137 - val_accuracy: 0.9058
Epoch 3/5
2123/2123 [==============================] - 160s 75ms/step - loss: 0.0204 - accuracy: 0.9929 - val_loss: 0.4722 - val_accuracy: 0.9195
Epoch 4/5
2123/2123 [==============================] - 159s 75ms/step - loss: 0.0191 - accuracy: 0.9937 - val_loss: 0.8227 - val_accuracy: 0.9178
Epoch 5/5
2123/2123 [==============================] - 160s 75ms/step - loss: 0.0196 - accuracy: 0.9939 - val_loss: 0.5556 - val_accuracy: 0.9227


## Checking results on the test set

In [ ]:
model.evaluate(np.array(X_test), np.array(y_test))

46/46 [==============================] - 1s 25ms/step - loss: 0.0525 - accuracy: 0.9808


[0.05253340303897858, 0.9807559847831726]